<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/SN_web_lightmode.png" width="300"> 
</center>

# Assignment: Exploratory Data Analysis with SQL
Estimated time needed: 60 minutes


# Introduction
Using this R notebook you will perform exploratory data analysis tasks using SQL queries with the RODBC R package. You will not be graded here, but you will be asked to present some of these results in your final assignment, where you will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your Db2 connection
Load the 'RODBC' library, and use the 'odbcConnect( )' function as you did in the previous lab to establish the connection to your Db2 assets.  
Provided you successfully loaded your data into the tables in that lab, you are now ready to start running SQL queries using the RODBC library as you did in Course 3.


In [7]:
library(RODBC)

In [26]:
# provide your solution here
# Establish database connection
dsn_driver <- "{IBM DB2 ODBC DRIVER}"
dsn_database <- "bludb"
dsn_hostname <- "b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_port <- "32304"
dsn_protocol <- "TCPIP"     
dsn_uid <- "tmd23140"   
dsn_pwd <- "PEGQQkAMktledIAK"   
dsn_security <- "ssl"
conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")
conn <- odbcDriverConnect(conn_path)

# Dump connection info
#############################################################
sql.info <- sqlTypeInfo(conn)
conn.info <- odbcGetInfo(conn)
conn.info["DBMS_Name"]
conn.info["DBMS_Ver"]
conn.info["Driver_ODBC_Ver"]

DBMS_Name 
"DB2/LINUXX8664"

DBMS_Ver 
"11.05.0800"

Driver_ODBC_Ver 
        "03.51"


## Task 1 - Record Count
#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [27]:
# provide your solution here
query = "SELECT COUNT(DATE) as record FROM seoul_bike_sharing ;"
view <- sqlQuery(conn,query)
view

,RECORD
,<int>
1,8465


## Task 2 - Operational Hours
#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [28]:
# provide your solution here
query = "SELECT COUNT(HOUR) as record FROM seoul_bike_sharing where RENTED_BIKE_COUNT <> 0;"
view <- sqlQuery(conn,query)
view

,RECORD
,<int>
1,8465


## Task 3 - Weather Outlook
#### Query the the weather forecast for Seoul over the next 3 hours.
Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [29]:
# provide your solution here
query = "SELECT * from cities_weather_forecast
Where CITY = 'Seoul'
Limit 1;"
view <- sqlQuery(conn,query)
view

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<fct>,<dttm>
1,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00


## Task 4 - Seasons
#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [30]:
# provide your solution here
query = "SELECT DISTINCT(SEASONS) as record FROM seoul_bike_sharing ;"
view <- sqlQuery(conn,query)
view

,RECORD
,<fct>
1,Autumn
2,Spring
3,Summer
4,Winter


## Task 5 - Date Range
#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [31]:
# provide your solution here
query = "SELECT min(DATE) as FIRST_DATE, max(DATE) as LAST_DATE FROM seoul_bike_sharing ;"
view <- sqlQuery(conn,query)
view

,FIRST_DATE,LAST_DATE
,<fct>,<fct>
1,01/01/2018,31/12/2017


## Task 6 - Subquery - 'all-time high'
#### determine which date and hour had the most bike rentals.


### Solution 6


In [32]:
# provide your solution here
query = "SELECT DATE, HOUR,RENTED_BIKE_COUNT FROM seoul_bike_sharing 
WHERE RENTED_BIKE_COUNT = (SELECT MAX(RENTED_BIKE_COUNT) from seoul_bike_sharing);"
view <- sqlQuery(conn,query)
view

,DATE,HOUR,RENTED_BIKE_COUNT
,<fct>,<int>,<int>
1,19/06/2018,18,3556


## Task 7 - Hourly popularity and temperature by season
#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [33]:
# provide your solution here
query = "SELECT SEASONS, HOUR, AVG(TEMPERATURE) as AVG_TEMP, AVG(RENTED_BIKE_COUNT) as AVG_RENT
FROM seoul_bike_sharing
GROUP by HOUR,SEASONS
ORDER BY AVG_RENT DESC
LIMIT 10;"
view <- sqlQuery(conn,query)
view

,SEASONS,HOUR,AVG_TEMP,AVG_RENT
,<fct>,<int>,<dbl>,<int>
1,Summer,18,29.38696,2135
2,Autumn,18,16.03086,1983
3,Summer,19,28.27283,1889
4,Summer,20,27.06630,1801
5,Summer,21,26.27826,1754
6,Spring,18,15.97222,1689
7,Summer,22,25.69891,1567
8,Autumn,17,17.27778,1562
9,Summer,17,30.07500,1526


## Task 8 - Rental Seasonality
#### Find the average hourly bike count during each season. 
Also include the minimum, maximum, and standard deviation of the hourly bike count for each season. 


### Solution 8


In [37]:
# provide your solution here
query = "SELECT SEASONS, HOUR, AVG(RENTED_BIKE_COUNT) as mean,
min(RENTED_BIKE_COUNT) as minn,
max(RENTED_BIKE_COUNT) as maxx,
STDDEV_POP(RENTED_BIKE_COUNT) as STD_l
FROM seoul_bike_sharing
GROUP by HOUR,SEASONS
ORDER BY mean DESC;"
view <- sqlQuery(conn,query)
view

,SEASONS,HOUR,MEAN,MINN,MAXX,STD_L
,<fct>,<int>,<int>,<int>,<int>,<dbl>
1,Summer,18,2135,17,3556,884.0829
2,Autumn,18,1983,40,3298,778.4414
3,Summer,19,1889,18,2984,728.8799
4,Summer,20,1801,10,2579,662.2163
5,Summer,21,1754,17,2505,596.1374
6,Spring,18,1689,22,3251,898.8971
7,Summer,22,1567,16,2309,516.6434
8,Autumn,17,1562,23,2432,554.3165
9,Summer,17,1526,25,2664,608.7917


Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality
#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?
Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all. 


### Solution 9


In [18]:
 # provide your solution here
query="SELECT SEASONS, AVG(rented_bike_count) as rented_bike_count,
AVG(TEMPERATURE)as TEMPERATURE, AVG(HUMIDITY) as HUMIDITY, AVG(WIND_SPEED) as WIND_SPEED, AVG(VISIBILITY) as VISIBILITY,
AVG(DEW_POINT_TEMPERATURE)as DEW_POINT_TEMPERATURE, AVG(SOLAR_RADIATION) as SOLAR_RADIATION, 
AVG(RAINFALL)as RAINFALL, AVG(SNOWFALL)as SNOWFALL
FROM seoul_bike_sharing
GROUP by SEASONS
ORDER BY rented_bike_count DESC;"
view <- sqlQuery(conn,query)
view

,SEASONS,RENTED_BIKE_COUNT,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL
,<fct>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,Summer,1034,26.587274,64,1.609420,1501,18.750136,0.7612545,0.25348732,0.00000000
2,Autumn,924,13.821167,59,1.492101,1558,5.150594,0.5227827,0.11765617,0.06350026
3,Spring,746,13.021389,58,1.857778,1240,4.091389,0.6803009,0.18694444,0.00000000
4,Winter,225,-2.540463,49,1.922685,1445,-12.416667,0.2981806,0.03282407,0.24750000


## Task 10 - Total Bike Count and City Info for Seoul
#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.
Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [19]:
# provide your solution here
query="SELECT A.CITY_ASCII, A.COUNTRY, A.LAT, A.LNG, A.POPULATION, B.BICYCLES
FROM world_cities A, bike_sharing_systems B
WHERE (A.CITY_ASCII= B.CITY) AND (A.CITY_ASCII = 'Seoul');"
view <- sqlQuery(conn,query)
view

,CITY_ASCII,COUNTRY,LAT,LNG,POPULATION,BICYCLES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Seoul,"Korea, South",37.58,127,21794000,20000


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system
#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.
Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [20]:
# provide your solution here
query="SELECT A.CITY_ASCII, A.COUNTRY, A.LAT, A.LNG, A.POPULATION, B.BICYCLES
FROM world_cities A, bike_sharing_systems B
WHERE (A.CITY_ASCII= B.CITY) AND (B.BICYCLES BETWEEN '15000' AND '20000');"
view <- sqlQuery(conn,query)
view

,CITY_ASCII,COUNTRY,LAT,LNG,POPULATION,BICYCLES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Shanghai,China,31.16,121.46,22120000,19165
2,Seoul,"Korea, South",37.58,127.00,21794000,20000
3,Beijing,China,39.90,116.39,19433000,16000
4,Weifang,China,36.71,119.10,9373000,20000
5,Heze,China,35.23,115.43,8750000,2000
6,Ningbo,China,29.87,121.54,7639000,15000
7,Xi'an,China,34.26,108.90,7135000,20000
8,Sydney,Australia,-33.86,151.20,5312163,2000
9,Zhuzhou,China,27.84,113.14,3855609,20000


In [21]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Yan Luo, Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                |
| ---------- | ------- | ------------- | --------------------------------------------------------------------------------- |
| 2021-05-04 | 0.2     | Jeff Grossman | Finalize                               |
| 2021-04-22 | 0.2     | Jeff Grossman | Update as per peer review                               |
| 2021-04-10 | 0.2     | Jeff Grossman | Add data sources and implicit joins                                  |
| 2021-04-05 | 0.1     | Jeff Grossman | Start content creation                                                          |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
